In [47]:
from GoogleNews import GoogleNews
import requests
import json
import pandas as pd
from openai import OpenAI
import os

In [40]:
def getNews(ticker_name):
    df = pd.read_csv('shap_top_features.csv')
    row = df[df['Ticker'] == ticker_name].iloc[0]
    d1, d2 = row['Date_1'], row['Date_2']
    f1, f2, f3, f4, f5 = row['Feature_1'], row['Feature_2'], row['Feature_3'], row['Feature_4'], row['Feature_5']
    googlenews = GoogleNews(start=d1, end=d2)
    googlenews.search(ticker_name)
    results = googlenews.results()
    headings = [article['title'] for article in results if 'title' in article]
    prompt_content = "\n".join(headings)
    prompt = (f"Given the following news headlines about {ticker_name} between {d1} and {d2}:\n"
              f"{prompt_content}\n\nPlease analyze based on features: {f1}, {f2}, {f3}, {f4}, {f5}. The Q4 in the features means just the last quarter and Q1 means the 4th last quarter. Explain the features with the news with your analysis to a beginner and always explain what is the meaning of each feature to very beginner and how does it afftect the stock movement between the d1 and d2 for that ticker showing any stark growth or fall or just normal gradual increase or decline.")

    return prompt

In [ ]:
def getSummary(Name='ZYDUSLIFE.NS'):
    prompt = getNews(Name)
    client = OpenAI(
        api_key="pplx-key",
        base_url="https://api.perplexity.ai"
    )
    
    resp = client.chat.completions.create(
        model="sonar",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return resp.choices[0].message.content

In [42]:
getSummary('ABB.NS')

'Here is an analysis of the ABB India (ABB.NS) news headlines and the reported quarterly financial features (EPS, shares, interest expense) between 2024-06-30 and 2025-06-30, explained for a beginner:\n\n---\n\n### Key Financial Features Explained Simply:\n\n- **Diluted EPS (Earnings Per Share):**  \n  This tells you how much profit the company made *per each share* after accounting for potentially convertible shares. If this number increases (+0.01), it means the company earned more profit per share, which is usually good for the stock price. If it decreases (-0.00), that indicates a slight loss impact on profit per share.\n\n- **Diluted Average Shares:**  \n  The average number of shares outstanding, adjusted for dilution. If this number rises (+0.00; almost stable), it means more shares, which could dilute (reduce) EPS unless net income rises proportionally.\n\n- **Interest Expense:**  \n  The cost the company pays on debt. If interest expense rises, it reduces net income; if it sta